**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests.

[Datasets](https://drive.google.com/drive/folders/1kf6BHlmF32UjsHw6wh_sMJ-JWkV091XD?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1XqPgu_rFZZ5eHCoFS7drFsmqd7U81vj_/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1aUJUdGm-RdHaJCJvS1kay9fRdNTq22xb/view?usp=sharing)

# 1. (15 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate.

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [ ]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from scipy import stats

from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
# Loading data
fb = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/facebook_ads.csv')

# Check the first lines
fb.head()


,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [ ]:
# Let's group by campaign ID and calculate the total number of impressions and clicks
sumstats = fb.groupby('xyz_campaign_id')['Impressions', 'Clicks'].sum().reset_index()


<ipython-input-32-da642725f48b>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sumstats = fb.groupby('xyz_campaign_id')['Impressions', 'Clicks'].sum().reset_index()


In [ ]:
# Calculate CTR (Click Through Rate) for each campaign
sumstats['ctr'] = sumstats['Clicks'] / sumstats['Impressions'] * 100


In [ ]:
# Calculate the standard deviation of CTR
sumstats['ctr_sd'] = np.sqrt(sumstats['ctr'] * (1 - sumstats['ctr'] / 100))


In [ ]:
# Determine the campaign with the highest CTR
sumstats['highest_ctr'] = sumstats['ctr'] == sumstats['ctr'].max()


In [ ]:
sumstats

,xyz_campaign_id,Impressions,Clicks,ctr,ctr_sd,highest_ctr
0,916,482925,113,0.023399,0.152950,False
1,936,8128187,1984,0.024409,0.156214,True
2,1178,204823716,36068,0.017609,0.132688,False


## Answer (edit this cell)
The campaign with the highest CTR is: **936**


# 2. (15 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget?

In [ ]:
# Calculate the 95% confidence interval for the CTR of each campaign
def compute_confidence_interval(row):
    mean = row['ctr']
    std_error = row['ctr_sd'] / np.sqrt(row['Impressions'])
    ci_lower, ci_upper = stats.norm.interval(0.95, loc=mean, scale=std_error)
    return pd.Series([ci_lower, ci_upper], index=['ci_lower', 'ci_upper'])


In [ ]:
# Apply the function to the dataframe
sumstats[['ci_lower', 'ci_upper']] = sumstats.apply(compute_confidence_interval, axis=1)


In [ ]:
# Function for two-sample t-test
def ttest_2sample(row1, row2):
    m1, sd1, N1 = row1['ctr'], row1['ctr_sd'], row1['Impressions']
    m2, sd2, N2 = row2['ctr'], row2['ctr_sd'], row2['Impressions']
    pval = stats.ttest_ind_from_stats(mean1=m1, std1=sd1, nobs1=N1,
                                      mean2=m2, std2=sd2, nobs2=N2,
                                      equal_var=False).pvalue
    return pval


In [ ]:
# Select lines for campaigns
campaign_916 = sumstats[sumstats['xyz_campaign_id'] == 916].iloc[0]
campaign_936 = sumstats[sumstats['xyz_campaign_id'] == 936].iloc[0]
campaign_1178 = sumstats[sumstats['xyz_campaign_id'] == 1178].iloc[0]


In [ ]:
# Two-sample t-tests
pval_936_vs_916 = ttest_2sample(campaign_936, campaign_916)
pval_936_vs_1178 = ttest_2sample(campaign_936, campaign_1178)


In [ ]:
sumstats[['xyz_campaign_id', 'ci_lower', 'ci_upper']], pval_936_vs_916, pval_936_vs_1178


(   xyz_campaign_id  ci_lower  ci_upper
 0              916  0.022968  0.023830
 1              936  0.024301  0.024516
 2             1178  0.017591  0.017627,
 8.501923365379338e-06,
 0.0)

**Edit this cell**

1. (The CTR confidence intervals for campaigns 916 and 936 show that campaign 936 has a slightly higher click-through rate compared to campaign 916. This is confirmed by the non-overlapping confidence intervals, where the upper bound of the interval for campaign 916 is lower than the lower bound of the interval for campaign 936, indicating to a statistically significant improvement in CTR for campaign 936.)
1. (Comparing the CTR confidence intervals for campaigns 916 and 1178 shows that campaign 916 has a higher click-through rate compared to campaign 1178, as evidenced by the non-overlapping intervals. The upper bound of the interval for campaign 916 is significantly higher than the lower bound of the interval for campaign 1178, which indicates that campaign 916 performed better in terms of CTR.)

In [ ]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):
    """
    This function requires you to supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2)
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))

    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [ ]:
# Get the necessary parameters from sumstats for campaigns
m1_936, sd1_936, N1_936 = campaign_936['ctr'], campaign_936['ctr_sd'], campaign_936['Impressions']
m2_916, sd2_916, N2_916 = campaign_916['ctr'], campaign_916['ctr_sd'], campaign_916['Impressions']
m2_1178, sd2_1178, N2_1178 = campaign_1178['ctr'], campaign_1178['ctr_sd'], campaign_1178['Impressions']

# Run a t-test to compare campaigns 936 and 916
pval_936_vs_916_function = ttest_2sample(m1_936, sd1_936, N1_936, m2_916, sd2_916, N2_916, twotail=True, equalvar=False)

# Run a t-test to compare campaigns 936 and 1178
pval_936_vs_1178_function = ttest_2sample(m1_936, sd1_936, N1_936, m2_1178, sd2_1178, N2_1178, twotail=True, equalvar=False)

pval_936_vs_916_function, pval_936_vs_1178_function


(8.501923365322739e-06, 0.0)

**Edit this cell to answer**

(Write a few sentences to describe how to proceed with the allocation of the ad budget going forward. Be sure to use the 2 tail t-tests above to support your strategy)

# 3. (20 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [ ]:
starbucks_promos = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/starbucks_promos.csv')

starbucks_promos.head()


,Unnamed: 0,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
0,1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
1,4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
2,5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
3,6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
4,8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [ ]:
# Differences in sales rates for BOGO and Discount promotions
bogo_redemption = starbucks_promos[starbucks_promos['offer_type'] == 'bogo']['redeemed']
discount_redemption = starbucks_promos[starbucks_promos['offer_type'] == 'discount']['redeemed']


In [ ]:
# t-test to compare redemption rates
ttest_result_redemption = stats.ttest_ind(bogo_redemption, discount_redemption)


In [ ]:
# Comparison of average income among those who took advantage of BOGO and Discount promotions
bogo_income = starbucks_promos[(starbucks_promos['offer_type'] == 'bogo') & (starbucks_promos['redeemed'] == 1) & (starbucks_promos['income'].notnull())]['income']
discount_income = starbucks_promos[(starbucks_promos['offer_type'] == 'discount') & (starbucks_promos['redeemed'] == 1) & (starbucks_promos['income'].notnull())]['income']

# t-test to compare average incomes
ttest_result_income = stats.ttest_ind(bogo_income, discount_income)


In [ ]:
ttest_result_redemption.pvalue, bogo_income.mean(), discount_income.mean(), ttest_result_income.pvalue


(6.611390792545181e-67,
 70114.8559593297,
 68617.39130434782,
 2.2763463492984577e-10)

**Edit this cell**
1. (A statistically significant difference in the implementation rates of BOGO and Discount promotions was found (p-value ≈ 6.61e-67), indicating a difference in the effectiveness of these two types of promotions.).
1. (Average revenue for BOGO promotions was $70,114.86, compared to $68,617.39 for Discount promotions. The test for equality of means showed a statistically significant difference (p-value ≈ 2.28e-10), indicating differences in revenue among customers who took advantage of different types of promotions.).